<a href="https://colab.research.google.com/github/patas-code/FaceValue_ML/blob/main/SyntheticBioFinaProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

0. Initialize all libraries


In [1]:
import os
import pandas as pd
from urllib.request import urlretrieve
#!pip install dnachisel
#!pip install synbiopython
#!pip install CodonTransformer
from dnachisel import *
from Bio.Seq import Seq
import synbiopython
import numpy
import torch
from transformers import AutoTokenizer, BigBirdForMaskedLM
from CodonTransformer.CodonPrediction import predict_dna_sequence
from CodonTransformer.CodonJupyter import format_model_output

#setting up CodonTransformer tool
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


import warnings
warnings.filterwarnings('ignore')

1. Get IDT DNA Sequence Optimized for Cpn60 sequence in the organisms Pseudonomas Putida:

Same plasmid & organism will be used as our coursework investigating sequence optimization to ensure there is no error in choosing or creating a new sequence. Since the website requires a login, we are inputting the file given by the website once run with our organism and desired chaperone/sequence.


In [3]:
# get file from website downloads (login-locked so use the raw output downlaoded file)
file = "H6OvrrA2.csv"
files= pd.read_csv(file)
seq=files.astype(str)
print(seq)
# ^^^^^ this printed output is the optimized sequence

Empty DataFrame
Columns: [Unnamed: 0, ATGGCGAAGGAAATCAAATTCGCAGAGGACGCCCGCGCCGCGATGCTGCGCGGGGTGGACGTGCTGGCCGATACCGTGAAAGTCACCCTCGGCCCGAAAGGCCGCAATGTCGTGCTCGAAAAATCGTTCGGCAGCCCGCTGATCACCAACGATGGCGTCACCATCGCCAAAGAAATCGAGCTGGAGGACCATTTCGAAAACATGGGCGCCAAGCTGGTGTCCGAAGTGGCGTCGAAGACCAACGACATCGCGGGGGATGGCACCACGACCGCGACGGTGCTGACCCAGGCGATCGTGCGTGAGGGCCTGAAGAACGTGACCGCCGGTGCCAACCCACTGGGCATCCGTCGCGGTATTGAGCTGGCGACCAAGACCGCCGTGGAAGAATTGCACAACATCTCCAGCGTGGTCGACAGCAAAGAAGCGATTGCCCAGGTCGCCGCAGTGAGCTCGGGCAGCGAAAAAGTGGGCCAACTGATCGCAGATGCTATGGAGAAGGTGGGCAACGACGGCGTCATCACCATCGAGGAATCGAAAGGCATCGAGACCGAGTTGGATGTCGTCGAAGGCATGCAGTTCGATCGCGGCTACTTGAGCCAGTATATGGTCACGGATAATGACAAAATGGAGGCTGTGCTGGAAAACCCGTACATCCTGATCACCGACAAAAAGATCTCCAACATCCAGGACATCCTGCCGTTGTTGGAACAGATCTTGCAGCAAAGCCGTCCGCTGCTGATCATCGCGGACGACGTGGATGGTGAGGCCCTGCCGACGCTGGTGCTGAACAAGATCCGTGGCACGTTTAACGTCGTCGCCGTCAAAGCCCCGGGGTTCGGGGACCGGCGTAAAGCGATGCTGGAGGACATCGCGATCCTGACCGGCGGGACGGTGATCACCGACGACTTGGGTCTGGAATTGAAGGACACGACCATTGAAAACTTGGGCAACGCCTCGAAAGT

In [4]:
seq1='ATGGCGAAGGAAATCAAATTCGCAGAGGACGCCCGCGCCGCGATGCTGCGCGGGGTGGACGTGCTGGCCGATACCGTGAAAGTCACCCTCGGCCCGAAAGGCCGCAATGTCGTGCTCGAAAAATCGTTCGGCAGCCCGCTGATCACCAACGATGGCGTCACCATCGCCAAAGAAATCGAGCTGGAGGACCATTTCGAAAACATGGGCGCCAAGCTGGTGTCCGAAGTGGCGTCGAAGACCAACGACATCGCGGGGGATGGCACCACGACCGCGACGGTGCTGACCCAGGCGATCGTGCGTGAGGGCCTGAAGAACGTGACCGCCGGTGCCAACCCACTGGGCATCCGTCGCGGTATTGAGCTGGCGACCAAGACCGCCGTGGAAGAATTGCACAACATCTCCAGCGTGGTCGACAGCAAAGAAGCGATTGCCCAGGTCGCCGCAGTGAGCTCGGGCAGCGAAAAAGTGGGCCAACTGATCGCAGATGCTATGGAGAAGGTGGGCAACGACGGCGTCATCACCATCGAGGAATCGAAAGGCATCGAGACCGAGTTGGATGTCGTCGAAGGCATGCAGTTCGATCGCGGCTACTTGAGCCAGTATATGGTCACGGATAATGACAAAATGGAGGCTGTGCTGGAAAACCCGTACATCCTGATCACCGACAAAAAGATCTCCAACATCCAGGACATCCTGCCGTTGTTGGAACAGATCTTGCAGCAAAGCCGTCCGCTGCTGATCATCGCGGACGACGTGGATGGTGAGGCCCTGCCGACGCTGGTGCTGAACAAGATCCGTGGCACGTTTAACGTCGTCGCCGTCAAAGCCCCGGGGTTCGGGGACCGGCGTAAAGCGATGCTGGAGGACATCGCGATCCTGACCGGCGGGACGGTGATCACCGACGACTTGGGTCTGGAATTGAAGGACACGACCATTGAAAACTTGGGCAACGCCTCGAAAGTCGTGGTGGACAAAGACAACACGACGATTGTCGAGGGCGCCGGGAGCAAAGAAGCGATCGATGCCCGCGTCCATCTGATCAAGAACCAGATCGGCGAAACCACGTCGGATTTCGACCGCGAGAAACTGCAAGAGCGCTTGGCGAAGCTCGCGGGCGGCGTGGCCGTCGTGAAGGTCGGGGCGGCCACCGAAACCGAACTGAAGGAACTGAAGCTGCGTATCGAGGATGCCTTGAACGCCACCCGCGCGGCGGTCGAAGAGGGCATGGTCAGCGGTGGTGGCACGGCTCTGGTCAACGTGATCGGCAAGGTGGCCGCGCTGGAGGCCGAAGGTGACGTGGCCACGGGCATCAAGATTGTGGTGCGTGCCCTGGAAGAGCCAATCCGTCAGATTGCGGAGAACGCCGGTTACGAAGGCAGCGTGATCGTGGACAAGTTGAAGAACGTGGACCTGGGCATCGGGTTCAATGCAGCTAACGGCGAATGGGTGAACATGGTGGAGGCTGGCATCGTGGACCCGACCAAAGTGACCCGGAGCGCCCTGCAGAATGCCGCCAGCGTCAGCGCCCTCCTGCTGACGACCGAGGCCGTCGTCGCCGACAAGCCGGAGCCGGCCGCGCCTGCTCCAATGATGGATCCGAGCATGGGCATGGGCGGGATGATG'

2. Get CodonTransformer Sequence Optimized for Cpn60 Sequence in the organisms Pseudonomas Putida:

In [5]:
#setting up neural network-driven codon optimizer
tokenizer = AutoTokenizer.from_pretrained("adibvafa/CodonTransformer")
model = BigBirdForMaskedLM.from_pretrained("adibvafa/CodonTransformer").to(device)

# Set your input data as per documentation
protein = "MAKDIKFSEDARRSMLRGVDKLADAVKVTLGPKGRNVVLEKKFGSPLITNDGVTIAKEIELEDAFENMGAKLVAEVASKTNDIAGDGTTTATVLAQAMIREGLKNVTSGANPMVIRKGIEKATQVAVEELSKISKPIEGKDSIAQVAAISSADDEVGKIIAEAMERVGNDGVITIEESKGFSTELEVVEGMQFDRGYASPYMVTDSDKMEAVLDNPYVLITDKKISNIQEVLPVLEQVVQQGKPILIIAEDVEGEALATLVVNKLRGTFNAVAVKAPGFGDRRKAMLEDIAILTGGEVITEDLGLDLKSANITQLGRASKVVVTKENTTIVEGAGESDKIAARVNQIKAQIEETTSDFDKEKLQERLAKLAGGVAVLKVGAATETEMKERKLRIEDALNSTRAAVEEGIVAGGGTALVNVIKAVSSIGAEGDEATGVNIVLRALEEPVRQIAHNAGLEGSVIVERLKKEEAGFGFNAATGEWVNMVEAGIVDPTKVTRSALQHAASVSAMFLTTEAVIADKPEENEGGGGMPDMGGMGGMGGMM"
organism = "Pseudomonas putida"

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/182 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/786 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/358M [00:00<?, ?B/s]

BigBirdForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
BigBirdForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`

In [ ]:
# Predict with CodonTransformer -> utilizes torch therefore requires GPU/CPU power
output = predict_dna_sequence(
    protein=protein,
    organism=organism,
    device=device,
    tokenizer=tokenizer,
    model=model,
    attention_type="original_full",
    deterministic=True
)
print(format_model_output(output))


-----------------------------
|          Organism         |
-----------------------------
Pseudomonas putida

-----------------------------
|       Input Protein       |
-----------------------------
MAKDIKFSEDARRSMLRGVDKLADAVKVTLGPKGRNVVLEKKFGSPLITNDGVTIAKEIELEDAFENMGAKLVAEVASKTNDIAGDGTTTATVLAQAMIREGLKNVTSGANPMVIRKGIEKATQVAVEELSKISKPIEGKDSIAQVAAISSADDEVGKIIAEAMERVGNDGVITIEESKGFSTELEVVEGMQFDRGYASPYMVTDSDKMEAVLDNPYVLITDKKISNIQEVLPVLEQVVQQGKPILIIAEDVEGEALATLVVNKLRGTFNAVAVKAPGFGDRRKAMLEDIAILTGGEVITEDLGLDLKSANITQLGRASKVVVTKENTTIVEGAGESDKIAARVNQIKAQIEETTSDFDKEKLQERLAKLAGGVAVLKVGAATETEMKERKLRIEDALNSTRAAVEEGIVAGGGTALVNVIKAVSSIGAEGDEATGVNIVLRALEEPVRQIAHNAGLEGSVIVERLKKEEAGFGFNAATGEWVNMVEAGIVDPTKVTRSALQHAASVSAMFLTTEAVIADKPEENEGGGGMPDMGGMGGMGGMM

-----------------------------
|      Processed Input      |
-----------------------------
M_UNK A_UNK K_UNK D_UNK I_UNK K_UNK F_UNK S_UNK E_UNK D_UNK A_UNK R_UNK R_UNK S_UNK M_UNK L_UNK R_UNK G_UNK V_UNK D_UNK K_UNK L_UNK A_UNK D_UNK A_UNK V_UNK K_UNK V_

In [6]:
seq2 = 'ATGGCCAAGGACATCAAGTTCAGCGAAGATGCCCGCCGCAGCATGCTGCGCGGCGTCGACAAGCTGGCCGATGCCGTCAAGGTCACCCTGGGCCCCAAGGGCCGCAACGTGGTGCTGGAGAAGAAGTTCGGCTCGCCGCTGATCACCAACGACGGCGTCACCATCGCCAAGGAGATCGAGCTGGAAGACGCCTTCGAAAACATGGGTGCCAAGCTGGTGGCCGAAGTGGCCAGCAAGACCAACGACATCGCCGGCGATGGCACCACCACCGCCACCGTGCTGGCCCAGGCCATGATCCGCGAAGGCCTGAAGAACGTCACCTCCGGTGCCAACCCGATGGTGATCCGCAAGGGCATCGAAAAGGCCACCCAGGTAGCCGTCGAAGAGCTGTCCAAGATCTCCAAGCCGATCGAAGGCAAGGACTCCATTGCCCAGGTGGCTGCCATCTCCTCGGCCGACGACGAAGTCGGCAAGATCATCGCCGAAGCCATGGAGCGCGTCGGCAACGACGGCGTGATCACCATCGAAGAGAGCAAGGGCTTCTCCACCGAGCTGGAAGTGGTCGAAGGCATGCAGTTCGACCGCGGCTATGCCAGCCCCTACATGGTCACCGACAGCGACAAGATGGAAGCGGTGCTGGACAACCCCTACGTGCTGATCACCGACAAGAAGATCTCCAACATCCAGGAAGTGCTGCCGGTGCTGGAGCAGGTGGTGCAGCAGGGCAAGCCGATCCTGATCATCGCCGAAGACGTCGAAGGCGAAGCGCTGGCCACCCTGGTGGTCAACAAGCTGCGTGGCACCTTCAACGCGGTGGCGGTGAAGGCACCTGGCTTCGGCGACCGCCGCAAGGCCATGCTGGAAGACATTGCCATCCTGACCGGCGGCGAAGTGATCACCGAAGACCTGGGCCTGGACCTGAAGTCGGCCAACATCACCCAGCTGGGCCGTGCCTCCAAGGTGGTGGTCACCAAGGAGAACACCACCATCGTCGAAGGTGCCGGCGAATCGGACAAGATTGCCGCCCGCGTCAACCAGATCAAGGCCCAGATCGAAGAAACCACCAGCGACTTCGACAAGGAAAAGCTGCAGGAGCGCCTGGCCAAGCTGGCCGGTGGCGTGGCGGTGCTGAAGGTCGGCGCCGCCACCGAAACCGAGATGAAGGAGCGCAAGCTGCGCATCGAAGATGCCCTGAACTCCACCCGCGCAGCGGTGGAAGAAGGCATCGTGGCCGGTGGTGGCACTGCGCTGGTCAACGTGATCAAGGCCGTGAGCTCGATCGGTGCCGAAGGCGACGAAGCCACCGGCGTCAACATCGTGCTGCGTGCCCTGGAAGAGCCGGTGCGCCAGATTGCCCACAACGCCGGCCTGGAAGGCTCGGTGATCGTCGAGCGCCTGAAGAAGGAAGAAGCCGGCTTCGGCTTCAACGCCGCCACCGGCGAATGGGTGAACATGGTGGAAGCCGGCATCGTCGACCCGACCAAGGTGACCCGCTCGGCCCTGCAGCACGCCGCCTCGGTGTCGGCCATGTTCCTGACCACCGAAGCGGTGATTGCCGACAAGCCAGAAGAGAACGAAGGCGGTGGCGGCATGCCAGACATGGGCGGCATGGGCGGCATGGGCGGCATGATGTGA'


3. Get DNA Chisel Sequence Optimized for Cpn60 sequence in the organisms Pseudonomas Putida:

In [7]:
# get Cpn60 sequence in organism optimized though DNA Chisel, no need for further initialization

# this sequence must be obtained from literature and put into the python library function, no way to just put nucleotides:
cpn60 = "ATGGCGGCTAAAGACGTGCTGTTTGGTGATTCTGCACGTGCGAAGATGTTGGTGGGTGTGAACATTCTGGCGGATGCCGTTCGTGTAACGCTGGGCCCGAAAGGTCGTAAC GTT GTC ATT GAA AAA TCA TTC GGC GCTCCG ATT ATC ACT AAA GAC GGC GTC AGC GTA GCC CGC GAA ATC GAGTTA AAG GAC AAA TTT GAG AAC ATG GGC GCG CAG ATG GTT AAA GAAGTA GCG AGC CAG GCC AAT GAT CAG GCG GGC GAT GGC ACA ACG ACCGCG ACC GTG CTG GCG CAG GCA ATT ATC AGC GAA GGA CTG AAG TCGGTT GCC GCG GGG ATG AAC CCC ATG GAT CTT AAA CGT GGT ATT GATAAA GCC ACC GCG GCA GTC GTG GCG GCC ATC AAA GAA CAG GCC CAACCG TGT CTG GAC ACT AAA GCG ATT GCC CAA GTG GGG ACA ATT TCAGCA AAC GCT GAT GAG ACC GTA GGC CGC TTA ATT GCC GAG GCG ATGGAA AAA GTG GGC AAG GAG GGC GTG ATC ACT GTG GAA GAA GGC AAA GGG CTG GAG GAC GAA CTG GAT GTT GTC GAA GGC ATG CAG TTT GAT CGC GGC TAC CTG AGT CCA TAC TTC ATC AAC AAC CAG GAA AAA ATG ACC GTG GAA ATG GAA AAT CCC CTT ATT CTG TTA GTT GAT AAA AAA ATC GAT AAT CTT CAG GAG CTT CTG CCG ATT CTT GAA AAT GTC GCA AAA TCT GGC CGC CCG CTG CTG ATC GTG GCG GAA GAT GTG GAA GGC CAG GCG TTG GCC ACC TTA GTT GTT AAT AAT CTG CGC GGG ACG TTC AAA GTG GCG GCC GTA AAA GCG CCG GGC TTC GGT GAC CGC CGT AAA GCT ATG CTT CAG GAT CTG GCG ATT CTG ACT GGT GGT CAG GTG ATC TCC GAA GAA CTG GGT ATG AGT TTG GAA ACG GCC GAT CCG TCA AGC CTT GGC ACC GCC TCC AAA GTT GTG ATC GAT AAA GAA AAC ACT GTG ATC GTC GAC GGT GCC GGC ACC GAA GCG TCG GTG AAC ACG CGC GTT GAT CAA ATC CGC GCT GAA ATT GAG TCG AGC ACC AGC GAC TAC GAT ATT GAA AAG CTG CAG GAG CGC GTT GCG AAA CTG GCA GGC GGC GTG GCG GTG ATC AAA GTA GGA GCA GGC AGT GAA ATG GAA ATG AAA GAA AAG AAG GAC CGC GTT GAT GAC GCC CTT CAC GCG ACG CGC GCG GCA GTT GAA GAA GGC GTC GTT GCC GGA GGT GGC GTG GCG CTG ATC CGT GCC CTG AGC TCT GTG ACC GTG GTA GGT GAT AAC GAA GAC CAA AAT GTG GGC ATC GCG CTT GCC CTT CGT GCG ATG GAA GCG CCG ATT CGT CAA ATT GCG GGG AAC GCC GGC GCG GAA GGC TCT GTT GTG GTG GAC AAG GTA AAG TCC GGC ACC GGG AGT TTC GGT TTT AAC GCG AGC ACC GGA GAA TAT GGA GAT ATG ATC GCA ATG GGG ATC CTG GAC CCG GCG AAA GTC ACC CGC AGC AGC CTG CAA GCC GCT GCG AGC ATT GCG GGC CTT ATG ATT ACC ACC GAA GCG ATG GTA GCG GAT GCT CCC GTT GAA GAG GGA GCA GGT GGA ATG CCC GAT ATG GGC GGC ATG GGC GGT ATG GGT GGT ATG CCT GGC ATG ATG "
cpn60 = cpn60.replace(" ","")
print(cpn60)

def optimize_for_pseudomonas_putida(sequence, output_file="optimized_sequence.fasta"):
    problem = DnaOptimizationProblem(
        sequence=cpn60,
        constraints=[
            EnforceTranslation(),
            AvoidHairpins(stem_size=10, hairpin_window=50),
            EnforceGCContent(mini=0.55, maxi=0.65, window=50),
            AvoidRareCodons(species='e_coli', min_frequency=0.1),  # Using E. coli as recommended by the documentation
        ],
        objectives=[
            CodonOptimize(species='e_coli', method='use_best_codon'),  # Use E. coli as proxy
        ]
    )

    problem.max_random_iters = 20000000
    problem.resolve_constraints()
    problem.optimize()
    print(problem.results())
    optimized_seq = problem.sequence

    # save as a fasta file, read in genbank (?)
    with open(output_file, 'w') as f:
        f.write(f">Optimized_Cpn60_Pseudomonas_putida\n{optimized_seq}")
    print(f"\nOptimizatoin saved as {output_file}")

    return optimized_seq

def main():
    # Optimize the sequence
    optimized_sequence = optimize_for_pseudomonas_putida(
        sequence=cpn60,
        output_file="cpn60_optimized_pputida.fasta"
    )
    print(optimized_sequence[:50] + "...")

if __name__ == "__main__":
    main()

ATGGCGGCTAAAGACGTGCTGTTTGGTGATTCTGCACGTGCGAAGATGTTGGTGGGTGTGAACATTCTGGCGGATGCCGTTCGTGTAACGCTGGGCCCGAAAGGTCGTAACGTTGTCATTGAAAAATCATTCGGCGCTCCGATTATCACTAAAGACGGCGTCAGCGTAGCCCGCGAAATCGAGTTAAAGGACAAATTTGAGAACATGGGCGCGCAGATGGTTAAAGAAGTAGCGAGCCAGGCCAATGATCAGGCGGGCGATGGCACAACGACCGCGACCGTGCTGGCGCAGGCAATTATCAGCGAAGGACTGAAGTCGGTTGCCGCGGGGATGAACCCCATGGATCTTAAACGTGGTATTGATAAAGCCACCGCGGCAGTCGTGGCGGCCATCAAAGAACAGGCCCAACCGTGTCTGGACACTAAAGCGATTGCCCAAGTGGGGACAATTTCAGCAAACGCTGATGAGACCGTAGGCCGCTTAATTGCCGAGGCGATGGAAAAAGTGGGCAAGGAGGGCGTGATCACTGTGGAAGAAGGCAAAGGGCTGGAGGACGAACTGGATGTTGTCGAAGGCATGCAGTTTGATCGCGGCTACCTGAGTCCATACTTCATCAACAACCAGGAAAAAATGACCGTGGAAATGGAAAATCCCCTTATTCTGTTAGTTGATAAAAAAATCGATAATCTTCAGGAGCTTCTGCCGATTCTTGAAAATGTCGCAAAATCTGGCCGCCCGCTGCTGATCGTGGCGGAAGATGTGGAAGGCCAGGCGTTGGCCACCTTAGTTGTTAATAATCTGCGCGGGACGTTCAAAGTGGCGGCCGTAAAAGCGCCGGGCTTCGGTGACCGCCGTAAAGCTATGCTTCAGGATCTGGCGATTCTGACTGGTGGTCAGGTGATCTCCGAAGAACTGGGTATGAGTTTGGAAACGGCCGATCCGTCAAGCCTTGGCACCGCCTCCAAAGTTGTGATCGATAAAGAAAACACTGTGATCGTCG

location:   0%|          | 0/29 [00:00<?, ?it/s, now=0-93]

KeyboardInterrupt: 

In [8]:
seq3='ATGGCGGCTAAAGACGTGCTGTTTGGTGATTCTGCACGTGCGAAGATGTTGGTGGGTGTGAACATTCTGGCGGATGCCGTTCGTGTAACGCTGGGCCCGAAAGGTCGTAACGTTGTCATTGAAAAATCATTCGGCGCTCCGATTATCACTAAAGACGGCGTCAGCGTAGCCCGCGAAATCGAGTTAAAGGACAAATTTGAGAACATGGGCGCGCAGATGGTTAAAGAAGTAGCGAGCCAGGCCAATGATCAGGCGGGCGATGGCACAACGACCGCGACCGTGCTGGCGCAGGCAATTATCAGCGAAGGACTGAAGTCGGTTGCCGCGGGGATGAACCCCATGGATCTTAAACGTGGTATTGATAAAGCCACCGCGGCAGTCGTGGCGGCCATCAAAGAACAGGCCCAACCGTGTCTGGACACTAAAGCGATTGCCCAAGTGGGGACAATTTCAGCAAACGCTGATGAGACCGTAGGCCGCTTAATTGCCGAGGCGATGGAAAAAGTGGGCAAGGAGGGCGTGATCACTGTGGAAGAAGGCAAAGGGCTGGAGGACGAACTGGATGTTGTCGAAGGCATGCAGTTTGATCGCGGCTACCTGAGTCCATACTTCATCAACAACCAGGAAAAAATGACCGTGGAAATGGAAAATCCCCTTATTCTGTTAGTTGATAAAAAAATCGATAATCTTCAGGAGCTTCTGCCGATTCTTGAAAATGTCGCAAAATCTGGCCGCCCGCTGCTGATCGTGGCGGAAGATGTGGAAGGCCAGGCGTTGGCCACCTTAGTTGTTAATAATCTGCGCGGGACGTTCAAAGTGGCGGCCGTAAAAGCGCCGGGCTTCGGTGACCGCCGTAAAGCTATGCTTCAGGATCTGGCGATTCTGACTGGTGGTCAGGTGATCTCCGAAGAACTGGGTATGAGTTTGGAAACGGCCGATCCGTCAAGCCTTGGCACCGCCTCCAAAGTTGTGATCGATAAAGAAAACACTGTGATCGTCGACGGTGCCGGCACCGAAGCGTCGGTGAACACGCGCGTTGATCAAATCCGCGCTGAAATTGAGTCGAGCACCAGCGACTACGATATTGAAAAGCTGCAGGAGCGCGTTGCGAAACTGGCAGGCGGCGTGGCGGTGATCAAAGTAGGAGCAGGCAGTGAAATGGAAATGAAAGAAAAGAAGGACCGCGTTGATGACGCCCTTCACGCGACGCGCGCGGCAGTTGAAGAAGGCGTCGTTGCCGGAGGTGGCGTGGCGCTGATCCGTGCCCTGAGCTCTGTGACCGTGGTAGGTGATAACGAAGACCAAAATGTGGGCATCGCGCTTGCCCTTCGTGCGATGGAAGCGCCGATTCGTCAAATTGCGGGGAACGCCGGCGCGGAAGGCTCTGTTGTGGTGGACAAGGTAAAGTCCGGCACCGGGAGTTTCGGTTTTAACGCGAGCACCGGAGAATATGGAGATATGATCGCAATGGGGATCCTGGACCCGGCGAAAGTCACCCGCAGCAGCCTGCAAGCCGCTGCGAGCATTGCGGGCCTTATGATTACCACCGAAGCGATGGTAGCGGATGCTCCCGTTGAAGAGGGAGCAGGTGGAATGCCCGATATGGGCGGCATGGGCGGTATGGGTGGTATGCCTGGCATGATG'

4. Comparing the optimized codon sequences:

In [9]:
# first check if they are the same 1-2, 2-3, 1-3?
if seq1==seq2:
  print('Optimization tools 1 and 2 output the same sequence.')
elif seq2==seq3:
  print('Optimization tools 2 and 3 output the same sequence.')
elif seq1==seq3:
  print('Optimization tools 1 and 3 output the same sequence.')
else:
  print('None of the sequences match! All optimizations are different.')

None of the sequences match! All optimizations are different.


In [11]:
# check how many codons/letter match between strings:
set1 = set(seq1.lower())
set2 = set(seq2.lower())
set3 = set(seq3.lower())

# The intersection (&) finds common elements; len() counts them
common_letters_count = len(set2 & set3 & set3)

print(f"Number of common unique letters: {common_letters_count}")


Number of common unique letters: 4
